In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [2]:
df = pd.read_csv('/Users/brandon/Desktop/MILO/test_files/dummy_train_column_scan.csv')
df = df.replace(r'^\s*$', np.nan, regex=True) #replaces empty strings spacess with NaN

In [3]:




enc = OneHotEncoder(handle_unknown='ignore')

#Need 2D vector to do the fit
vector = invalid[col_data['name']].values.reshape(-1,1)
enc.fit(vector)

trans = enc.transform(vector).toarray()

temp_df = pd.DataFrame(trans, columns=enc.categories_).add_prefix(col_data['name'] + '_').astype('int')

#Adjust original columns
invalid = pd.concat([temp_df,invalid], axis=1)
invalid = invalid.drop(col_data['name'], axis=1)

NameError: name 'invalid' is not defined

In [20]:
df['va_c'].name

'va_c'

In [23]:
def one_hot_encode(series):
    working = pd.Series(series)
    working = working.dropna()
    index = working.index
    
    enc = OneHotEncoder(handle_unknown='ignore')
    vector = working.values.reshape(-1,1)
    enc.fit(vector)
    trans = enc.transform(vector).toarray()
    output = pd.DataFrame(trans, columns=enc.categories_, index=index).add_prefix(series.name + '_').astype('int')
    
    return output

one_hot_encode(df['va_c'])

,va_c_0.044,va_c_0.061,va_c_0.063,va_c_0.171,va_c_0.199,va_c_0.252,va_c_0.324,va_c_0.36,va_c_0.52,va_c_0.596,va_c_0.727,va_c_0.75,va_c_0.916,va_c_0.941,va_c_0.944,va_c_a,va_c_b
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
7,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
